# Lab 03: Face Service - Face Detection and Analysis

## Introduction

The Azure AI Face service provides AI algorithms that detect, recognize, and analyze human faces in images. Face detection is a key capability that enables many scenarios, from access control to demographics analysis.

In this lab, you will:
- Detect faces in images using the Azure Face API
- Analyze face attributes such as age, emotion, accessories, and more
- Visualize detected faces with bounding boxes
- Understand the data returned by the Face API

## Prerequisites

- Azure subscription with Azure AI Services resource
- Python 3.7 or later
- Required packages: azure-cognitiveservices-vision-face, python-dotenv, matplotlib, pillow

## Setup: Install Required Packages

First, let's install the necessary packages for working with the Azure Face API.

In [ ]:
# Install required packages
!pip install azure-cognitiveservices-vision-face python-dotenv matplotlib pillow --quiet

## Step 1: Import Required Libraries

Import all the libraries we'll need for face detection and analysis.

In [ ]:
from dotenv import load_dotenv
import os
from PIL import Image, ImageDraw, ImageFont
from matplotlib import pyplot as plt
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.face.models import FaceAttributeType
from msrest.authentication import CognitiveServicesCredentials
import numpy as np

print('Libraries imported successfully!')

## Step 2: Load Configuration Settings

Load the Azure AI Services endpoint and key from the `.env` file. Make sure you've created a `.env` file in the `python/face-api/` directory with your credentials:

```
AI_SERVICE_ENDPOINT=your_ai_services_endpoint
AI_SERVICE_KEY=your_ai_services_key
```

In [ ]:
# Load environment variables from .env file
load_dotenv('python/face-api/.env')

# Get Azure AI Services endpoint and key
cog_endpoint = os.getenv('AI_SERVICE_ENDPOINT')
cog_key = os.getenv('AI_SERVICE_KEY')

# Verify credentials are loaded
if cog_endpoint and cog_key:
    print(f'✓ Endpoint loaded: {cog_endpoint}')
    print('✓ API key loaded successfully')
else:
    print('⚠️  Warning: Credentials not found. Please check your .env file.')

## Step 3: Authenticate Face Client

Create a Face client using the endpoint and key. This client will be used to make API calls to the Azure Face service.

In [ ]:
# Create credentials object
credentials = CognitiveServicesCredentials(cog_key)

# Create Face client
face_client = FaceClient(cog_endpoint, credentials)

print('✓ Face client authenticated successfully!')

## Step 4: Basic Face Detection

Let's start with the simplest use case - detecting faces in an image. We'll use the `detect_with_stream` method which returns basic face information including the bounding box location.

In [ ]:
# Specify the image file
image_file = 'python/face-api/images/face1.jpg'

# Display the original image
print('Analyzing image:', image_file)
img = Image.open(image_file)
plt.figure(figsize=(8, 6))
plt.axis('off')
plt.imshow(img)
plt.title('Original Image')
plt.show()

# Detect faces (basic detection without attributes)
with open(image_file, 'rb') as image_stream:
    detected_faces = face_client.face.detect_with_stream(image_stream)

print(f'\n✓ Found {len(detected_faces)} face(s) in the image')

### Understanding Face Detection Results

Each detected face includes:
- **face_id**: A unique identifier for the detected face (valid for 24 hours)
- **face_rectangle**: The bounding box coordinates (left, top, width, height)

Let's examine the detected face data:

In [ ]:
# Display details of detected faces
for i, face in enumerate(detected_faces, 1):
    print(f'\nFace {i}:')
    print(f'  Face ID: {face.face_id}')
    rect = face.face_rectangle
    print(f'  Location: Left={rect.left}, Top={rect.top}')
    print(f'  Size: Width={rect.width}, Height={rect.height}')

## Step 5: Visualize Detected Faces

Now let's visualize the detected faces by drawing bounding boxes around them.

In [ ]:
def visualize_faces(image_file, detected_faces, title='Detected Faces'):
    """
    Visualize detected faces with bounding boxes.
    
    Args:
        image_file: Path to the image file
        detected_faces: List of detected face objects from Face API
        title: Title for the plot
    """
    # Open image and create drawing context
    img = Image.open(image_file)
    draw = ImageDraw.Draw(img)
    
    # Define colors for bounding boxes
    color = 'lightgreen'
    
    # Draw a bounding box around each face
    for i, face in enumerate(detected_faces, 1):
        rect = face.face_rectangle
        left = rect.left
        top = rect.top
        right = left + rect.width
        bottom = top + rect.height
        
        # Draw rectangle
        draw.rectangle([(left, top), (right, bottom)], outline=color, width=5)
        
        # Add face number label
        draw.text((left, top - 20), f'Face {i}', fill=color)
    
    # Display the annotated image
    plt.figure(figsize=(10, 8))
    plt.axis('off')
    plt.imshow(img)
    plt.title(title)
    plt.show()
    
    return img

# Visualize the detected faces
annotated_img = visualize_faces(image_file, detected_faces)

## Step 6: Face Detection with Attributes

The Face API can analyze various face attributes beyond just detection. Let's detect faces and request specific attributes:

- **Age**: Estimated age of the person
- **Gender**: Detected gender
- **Emotion**: Emotional state (happiness, sadness, anger, etc.)
- **Glasses**: Type of glasses (if any)
- **Hair**: Hair color and baldness
- **Facial Hair**: Presence of mustache, beard, or sideburns
- **Accessories**: Accessories like glasses, masks, etc.
- **Makeup**: Presence of eye or lip makeup
- **Smile**: Smile intensity
- **Head Pose**: Head orientation (pitch, roll, yaw)

In [ ]:
# Specify the face attributes we want to analyze
face_attributes = [
    FaceAttributeType.age,
    FaceAttributeType.gender,
    FaceAttributeType.emotion,
    FaceAttributeType.glasses,
    FaceAttributeType.hair,
    FaceAttributeType.facial_hair,
    FaceAttributeType.accessories,
    FaceAttributeType.makeup,
    FaceAttributeType.smile,
    FaceAttributeType.head_pose
]

# Detect faces with attributes
print(f'Analyzing faces with attributes in: {image_file}')
with open(image_file, 'rb') as image_stream:
    detected_faces_with_attrs = face_client.face.detect_with_stream(
        image_stream,
        return_face_attributes=face_attributes
    )

print(f'\n✓ Detected {len(detected_faces_with_attrs)} face(s) with attributes')

## Step 7: Analyze Face Attributes

Let's examine the detailed attributes for each detected face.

In [ ]:
def analyze_face_attributes(face, face_number):
    """
    Display detailed analysis of face attributes.
    
    Args:
        face: Detected face object with attributes
        face_number: Face number for display
    """
    print(f'\n{"="*60}')
    print(f'FACE {face_number} ANALYSIS')
    print(f'{"="*60}')
    
    attrs = face.face_attributes
    
    # Basic demographics
    print(f'\n📊 Demographics:')
    print(f'   Age: {attrs.age} years old')
    print(f'   Gender: {attrs.gender}')
    
    # Emotions
    print(f'\n😊 Emotions:')
    emotions = {
        'Happiness': attrs.emotion.happiness,
        'Sadness': attrs.emotion.sadness,
        'Anger': attrs.emotion.anger,
        'Contempt': attrs.emotion.contempt,
        'Disgust': attrs.emotion.disgust,
        'Fear': attrs.emotion.fear,
        'Surprise': attrs.emotion.surprise,
        'Neutral': attrs.emotion.neutral
    }
    
    # Find dominant emotion
    dominant_emotion = max(emotions, key=emotions.get)
    print(f'   Dominant Emotion: {dominant_emotion} ({emotions[dominant_emotion]:.2%})')
    
    # Display all emotions
    print(f'   All Emotions:')
    for emotion, value in sorted(emotions.items(), key=lambda x: x[1], reverse=True):
        bar = '█' * int(value * 20)
        print(f'      {emotion:12s}: {bar} {value:.2%}')
    
    # Facial features
    print(f'\n👓 Accessories & Features:')
    print(f'   Glasses: {attrs.glasses}')
    print(f'   Smile: {attrs.smile:.2%}')
    
    # Hair
    print(f'\n💇 Hair:')
    if attrs.hair.hair_color:
        print(f'   Hair Colors:')
        for color in sorted(attrs.hair.hair_color, key=lambda x: x.confidence, reverse=True):
            print(f'      {color.color}: {color.confidence:.2%}')
    print(f'   Bald: {attrs.hair.bald:.2%}')
    
    # Facial hair
    print(f'\n🧔 Facial Hair:')
    print(f'   Mustache: {attrs.facial_hair.moustache:.2%}')
    print(f'   Beard: {attrs.facial_hair.beard:.2%}')
    print(f'   Sideburns: {attrs.facial_hair.sideburns:.2%}')
    
    # Makeup
    print(f'\n💄 Makeup:')
    print(f'   Eye Makeup: {"Yes" if attrs.makeup.eye_makeup else "No"}')
    print(f'   Lip Makeup: {"Yes" if attrs.makeup.lip_makeup else "No"}')
    
    # Accessories
    print(f'\n👑 Accessories:')
    if attrs.accessories:
        for accessory in attrs.accessories:
            print(f'   {accessory.type}: {accessory.confidence:.2%}')
    else:
        print(f'   None detected')
    
    # Head pose
    print(f'\n🔄 Head Pose:')
    print(f'   Pitch: {attrs.head_pose.pitch:.1f}°')
    print(f'   Roll: {attrs.head_pose.roll:.1f}°')
    print(f'   Yaw: {attrs.head_pose.yaw:.1f}°')

# Analyze each detected face
for i, face in enumerate(detected_faces_with_attrs, 1):
    analyze_face_attributes(face, i)

## Step 8: Visualize Faces with Attribute Labels

Let's create an enhanced visualization that shows key attributes alongside the bounding boxes.

In [ ]:
def visualize_faces_with_attributes(image_file, detected_faces):
    """
    Visualize detected faces with attribute labels.
    """
    # Open image
    img = Image.open(image_file)
    draw = ImageDraw.Draw(img)
    
    # Process each face
    for i, face in enumerate(detected_faces, 1):
        rect = face.face_rectangle
        left = rect.left
        top = rect.top
        right = left + rect.width
        bottom = top + rect.height
        
        # Draw bounding box
        draw.rectangle([(left, top), (right, bottom)], outline='lightgreen', width=5)
        
        # Get attributes
        attrs = face.face_attributes
        
        # Find dominant emotion
        emotions = {
            'Happy': attrs.emotion.happiness,
            'Sad': attrs.emotion.sadness,
            'Angry': attrs.emotion.anger,
            'Neutral': attrs.emotion.neutral
        }
        dominant_emotion = max(emotions, key=emotions.get)
        
        # Create label text
        label_lines = [
            f'Face {i}',
            f'Age: {int(attrs.age)}',
            f'{attrs.gender}',
            f'{dominant_emotion}',
            f'{attrs.glasses}'
        ]
        
        # Draw labels
        y_offset = top - 90
        for line in label_lines:
            draw.text((left, y_offset), line, fill='lightgreen')
            y_offset += 15
    
    # Display
    plt.figure(figsize=(12, 10))
    plt.axis('off')
    plt.imshow(img)
    plt.title('Face Detection with Attributes')
    plt.show()
    
    return img

# Visualize faces with attributes
annotated_img = visualize_faces_with_attributes(image_file, detected_faces_with_attrs)

## Step 9: Analyze Multiple Images

Let's try analyzing different images to see how the Face API handles various scenarios.

In [ ]:
# List of available images
image_files = [
    'python/face-api/images/face1.jpg',
    'python/face-api/images/face2.jpg',
    'python/face-api/images/faces.jpg'
]

# Process each image
for image_file in image_files:
    if os.path.exists(image_file):
        print(f'\n{"#"*70}')
        print(f'Processing: {image_file}')
        print(f'{"#"*70}')
        
        # Detect faces
        with open(image_file, 'rb') as image_stream:
            faces = face_client.face.detect_with_stream(
                image_stream,
                return_face_attributes=[FaceAttributeType.age, 
                                       FaceAttributeType.gender,
                                       FaceAttributeType.emotion]
            )
        
        print(f'Found {len(faces)} face(s)')
        
        # Visualize
        if faces:
            visualize_faces_with_attributes(image_file, faces)
    else:
        print(f'Image not found: {image_file}')

## Step 10: Emotion Analysis Deep Dive

Let's create a detailed emotion analysis visualization to better understand the emotional state detected in faces.

In [ ]:
def plot_emotion_chart(face, face_number):
    """
    Create a bar chart showing emotion scores for a face.
    """
    emotions = face.face_attributes.emotion
    
    emotion_dict = {
        'Happiness': emotions.happiness,
        'Neutral': emotions.neutral,
        'Sadness': emotions.sadness,
        'Anger': emotions.anger,
        'Surprise': emotions.surprise,
        'Fear': emotions.fear,
        'Contempt': emotions.contempt,
        'Disgust': emotions.disgust
    }
    
    # Sort by value
    sorted_emotions = dict(sorted(emotion_dict.items(), key=lambda x: x[1], reverse=True))
    
    # Create bar chart
    plt.figure(figsize=(10, 6))
    colors = ['#2ecc71' if v == max(sorted_emotions.values()) else '#3498db' 
              for v in sorted_emotions.values()]
    plt.barh(list(sorted_emotions.keys()), list(sorted_emotions.values()), color=colors)
    plt.xlabel('Confidence Score')
    plt.title(f'Emotion Analysis - Face {face_number}')
    plt.xlim(0, 1)
    
    # Add value labels
    for i, (emotion, value) in enumerate(sorted_emotions.items()):
        plt.text(value + 0.02, i, f'{value:.2%}', va='center')
    
    plt.tight_layout()
    plt.show()

# Plot emotion chart for first face
if detected_faces_with_attrs:
    plot_emotion_chart(detected_faces_with_attrs[0], 1)

## Summary

In this lab, you learned how to:

✅ Set up and authenticate the Azure Face API client  
✅ Detect faces in images and get bounding box coordinates  
✅ Analyze comprehensive face attributes including age, gender, emotion, and more  
✅ Visualize detected faces with bounding boxes and labels  
✅ Create detailed emotion analysis charts  

## Key Takeaways

- The Face API can detect multiple faces in a single image
- Face attributes include demographics, emotions, accessories, and physical features
- Emotion detection returns confidence scores for 8 different emotions
- Face IDs are temporary (24 hours) and used for comparison operations
- The API provides rich data that can be used for various AI applications

## Next Steps

Continue to the **Advanced Topics Notebook** to learn about:
- Face comparison and verification
- Finding similar faces
- Face grouping
- Detailed facial landmarks
- Face recognition best practices

## Cleanup (Optional)

If needed, you can save the annotated images to disk.

In [ ]:
# Save the last annotated image
if 'annotated_img' in locals():
    output_file = 'detected_faces_output.jpg'
    annotated_img.save(output_file)
    print(f'✓ Annotated image saved to: {output_file}')